In [5]:
import torch 
import numpy as np
from torch.nn import functional as F

def euclidean_distance(qf, gf):
    m = qf.shape[0]
    n = gf.shape[0]
    dist_mat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
               torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
    dist_mat.addmm_(1, -2, qf, gf.t())
    return dist_mat.cpu().numpy()

'''
A quantitative metric for Identity density to replce visualization tools 
like t-SNE, which is random and only focus on few samples.
'''
def ID2(feats, pid):
    feats = F.normalize(feats , dim=1, p=2)
    pids = np.asarray(pid)

    id_set = set(pids)
    id_list = list(id_set)
    id_list.sort()
    id_center = []
    for i in id_list:
        mask = pids == i
        x = feats[mask].mean(dim=0)
        id_center.append(x)

    density = torch.zeros(feats.size(0))
    idx = 0
    for i in id_list:
        mask = pids == i
        center = id_center[idx].unsqueeze(0)
        density[mask] = torch.tensor(euclidean_distance(feats[mask], center)).squeeze(1)
        idx += 1
    return density


In [ ]:
import os
from config import cfg
import argparse
from datasets import make_dataloader
from model import make_model
from processor import do_inference
from utils.logger import setup_logger


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="ReID Baseline Training")
    parser.add_argument(
        "--config_file", default="", help="path to config file", type=str
    )
    parser.add_argument("opts", help="Modify config options using the command-line", default=None,
                        nargs=argparse.REMAINDER)

    args = parser.parse_args()



    if args.config_file != "":
        cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()

    output_dir = cfg.OUTPUT_DIR
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    logger = setup_logger("transreid", output_dir, if_train=False)
    logger.info(args)

    if args.config_file != "":
        logger.info("Loaded configuration file {}".format(args.config_file))
        with open(args.config_file, 'r') as cf:
            config_str = "\n" + cf.read()
            logger.info(config_str)
    logger.info("Running with config:\n{}".format(cfg))

    os.environ['CUDA_VISIBLE_DEVICES'] = cfg.MODEL.DEVICE_ID

    train_loader, train_loader_normal, val_loader, num_query, num_classes, camera_num, view_num = make_dataloader(cfg)

    model = make_model(cfg, num_class=num_classes, camera_num=camera_num, view_num = view_num)
    model.load_param(cfg.TEST.WEIGHT)

    if cfg.DATASETS.NAMES == 'VehicleID':
        for trial in range(10):
            train_loader, train_loader_normal, val_loader, num_query, num_classes, camera_num, view_num = make_dataloader(cfg)
            rank_1, rank5 = do_inference(cfg,
                 model,
                 val_loader,
                 num_query)
            if trial == 0:
                all_rank_1 = rank_1
                all_rank_5 = rank5
            else:
                all_rank_1 = all_rank_1 + rank_1
                all_rank_5 = all_rank_5 + rank5

            logger.info("rank_1:{}, rank_5 {} : trial : {}".format(rank_1, rank5, trial))
        logger.info("sum_rank_1:{:.1%}, sum_rank_5 {:.1%}".format(all_rank_1.sum()/10.0, all_rank_5.sum()/10.0))
    else:
       do_inference(cfg,
                 model,
                 val_loader,
                 num_query)


In [6]:
# Simulate fake feature vectors (10 people, 512-dimensional features)
num_samples = 10
feature_dim = 512
torch.manual_seed(0)  # for reproducibility
feats = torch.randn(num_samples, feature_dim)

# Simulate person IDs: 5 classes, 2 samples per person
pid = [i // 2 for i in range(num_samples)]  # [0,0,1,1,2,2,3,3,4,4]

# Test the ID2 function
density_scores = ID2(feats, pid)

# Print results
print("Feature vector shape:", feats.shape)
print("Person IDs:", pid)
print("Density scores:", density_scores)


Feature vector shape: torch.Size([10, 512])
Person IDs: [0, 0, 1, 1, 2, 2, 3, 3, 4, 4]
Density scores: tensor([0.5327, 0.5327, 0.4876, 0.4876, 0.5300, 0.5300, 0.5116, 0.5116, 0.4855,
        0.4855])


C:\Users\Luis Mendez\AppData\Local\Temp\ipykernel_33348\3371697051.py:10: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta = 1, Number alpha = 1) (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\python_arg_parser.cpp:1661.)
  dist_mat.addmm_(1, -2, qf, gf.t())
